<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>

# Working with a real world data-set using SQL and Python

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Understand the dataset for Chicago Public School level performance
*   Store the dataset in SQLite database.
*   Retrieve metadata about tables and columns and query data from mixed case columns
*   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&download=true&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**:

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01">link</a>.

Now review some of its contents.


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database

##### The syntax for connecting to magic sql using sqllite is

  **%sql sqlite://DatabaseName**   
  
where DatabaseName will be your **.db** file 


In [1]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [2]:
# !pip install pandas
# !pip install ipython-sql prettytable
import pandas as pd

import prettytable
prettytable.DEFAULT = 'DEFAULT'

In [3]:
# !pip install ipython-sql
%load_ext sql

In [4]:
%sql sqlite:///RealWorldData.db

### Store the dataset in a Table

##### In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet.  
##### To analyze the data using SQL, it first needs to be stored in the database.

##### We will first read the csv files  from the given url  into pandas dataframes

##### Next we will be using the  df.to_sql() function to convert each csv file  to a table in sqlite  with the csv data loaded in it.


In [5]:
df = pd.read_csv('ChicagoPublicSchools.csv')

df.to_sql(name='CHICAGO_PUBLIC_SCHOOLS_DATA', con=con, if_exists='replace', index=False)

566

Double-click **here** for the solution.
<!--

import pandas

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

-->


### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [6]:
# type in your query to retrieve list of all tables in the database
# %sql select name from sqlite_master where type = 'table';

cur.execute("select name from sqlite_master where type='table'")

tables = cur.fetchall()

print("Tables in the DB:")
for i, table in enumerate(tables, start=1):
    print(f"{i}. {table[0]}")

Tables in the DB:
1. CENSUS_DATA
2. CHICAGO_PUBLIC_SCHOOLS_DATA


Double-click **here** for a hint

<!--
In sqlite the system catalog table called sqlite_master contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution using SQLite:

%sql SELECT name FROM sqlite_master WHERE type='table'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [7]:
# type in your query to retrieve the number of columns in the SCHOOLS table
# %sql SELECT count(name) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');
cur.execute("pragma table_info(CHICAGO_PUBLIC_SCHOOLS_DATA)")

columns = cur.fetchall()

print(f'Number of columns: {len(columns)}')

Number of columns: 78


Double-click **here** for the solution.

<!-- Solution SQLite:
%sql SELECT count(name) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.

In [8]:
from prettytable import PrettyTable

# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
cur.execute("pragma table_info(CHICAGO_PUBLIC_SCHOOLS_DATA);")
# %sql SELECT name, type, length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

table = PrettyTable()
table.field_names = ["name", "type", "length"]

for row in cur.fetchall():
    name = row[1]
    dtype = row[2]
    length = len(dtype)
    table.add_row([name, dtype, length])

print(table)

+--------------------------------------------------+---------+--------+
|                       name                       |   type  | length |
+--------------------------------------------------+---------+--------+
|                    School_ID                     | INTEGER |   7    |
|                  NAME_OF_SCHOOL                  |   TEXT  |   4    |
|        Elementary, Middle, or High School        |   TEXT  |   4    |
|                  Street_Address                  |   TEXT  |   4    |
|                       City                       |   TEXT  |   4    |
|                      State                       |   TEXT  |   4    |
|                     ZIP_Code                     | INTEGER |   7    |
|                   Phone_Number                   |   TEXT  |   4    |
|                       Link                       |   TEXT  |   4    |
|                 Network_Manager                  |   TEXT  |   4    |
|                Collaborative_Name                |   TEXT  |  

Double-click **here** for the solution.

<!-- Solution 

%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


### Answers:
### 1. The column name for the `"SCHOOL ID"` attribute in mixed case
### 2. The actual column name is `COMMUNITY_AREA_NAME`. It does not have spaces. Spaces were replaced by underscores.
### 3. Yes, several column names that originally had spaces or parentheses have been replaced with underscores.
    The column Rate_of_Misconducts__per_100_students_ likely comes from a name like:
    `"Rate of Misconducts (per 100 students)"`.

    Parentheses are removed, and their content is wrapped with extra underscores (e.g., `__per_100_students_`).

    However, not all columns follow this rule. For example:

`9th Grade EXPLORE (2009)` is still written exactly like that, with spaces and parentheses, meaning it was explicitly preserved during table creation (probably enclosed in double quotes).

## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [9]:
# %sql select count(*) from CHICAGO_PUBLIC_SCHOOLS_DATA where [Elementary, Middle, or High School] = 'ES';

query = """
select count(*)
from CHICAGO_PUBLIC_SCHOOLS_DATA
where [Elementary, Middle, or High School] = 'ES';
"""

cur.execute(query)
result = cur.fetchone()[0]
print(f"Count of Elementary schools: {result}")

Count of Elementary schools: 462


Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from CHICAGO_PUBLIC_SCHOOLS_DATA where "Elementary, Middle, or High School"='ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [10]:
# %sql select max(SAFETY_SCORE) as max_safety_score from CHICAGO_PUBLIC_SCHOOLS_DATA;

cur.execute("select max(SAFETY_SCORE) as max_safety_score from CHICAGO_PUBLIC_SCHOOLS_DATA;")
max_score = cur.fetchone()[0]
print(f"Max Safety Score: {max_score}")

Max Safety Score: 99.0


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from CHICAGO_PUBLIC_SCHOOLS_DATA

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [11]:
# %%sql 
# select NAME_OF_SCHOOL, SAFETY_SCORE
# from CHICAGO_PUBLIC_SCHOOLS_DATA
# where SAFETY_SCORE = (
#     select max(SAFETY_SCORE)
#     from CHICAGO_PUBLIC_SCHOOLS_DATA
# );

query = """
select NAME_OF_SCHOOL, SAFETY_SCORE
from CHICAGO_PUBLIC_SCHOOLS_DATA
where SAFETY_SCORE = (
    select max(SAFETY_SCORE)
    from CHICAGO_PUBLIC_SCHOOLS_DATA
);
"""

cur.execute(query)

rows = cur.fetchall()

# for name, score in rows:
#     print(f"{name}: {score}")

column_names = [desc[0] for desc in cur.description]

school_df = pd.DataFrame(rows, columns=column_names)

display(school_df)

,NAME_OF_SCHOOL,SAFETY_SCORE
0,Abraham Lincoln Elementary School,99.0
1,Alexander Graham Bell Elementary School,99.0
2,Annie Keller Elementary Gifted Magnet School,99.0
3,Augustus H Burley Elementary School,99.0
4,Edgar Allan Poe Elementary Classical School,99.0
5,Edgebrook Elementary School,99.0
6,Ellen Mitchell Elementary School,99.0
7,James E McDade Elementary Classical School,99.0
8,James G Blaine Elementary School,99.0
9,LaSalle Elementary Language Academy,99.0


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from CHICAGO_PUBLIC_SCHOOLS_DATA where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from CHICAGO_PUBLIC_SCHOOLS_DATA where \
  Safety_Score= (select MAX(Safety_Score) from CHICAGO_PUBLIC_SCHOOLS_DATA)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [12]:
# %%sql
# select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE
# from CHICAGO_PUBLIC_SCHOOLS_DATA
# order by AVERAGE_STUDENT_ATTENDANCE desc
# limit 10;

query = """
select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE
from CHICAGO_PUBLIC_SCHOOLS_DATA
order by AVERAGE_STUDENT_ATTENDANCE desc
limit 10
"""
cur.execute(query)

rows = cur.fetchall()

column_names = [desc[0] for desc in cur.description]

attendance_df = pd.DataFrame(rows, columns=column_names)

display(attendance_df)

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
0,John Charles Haines Elementary School,98.40%
1,James Ward Elementary School,97.80%
2,Edgar Allan Poe Elementary Classical School,97.60%
3,Orozco Fine Arts & Sciences Elementary School,97.60%
4,Rachel Carson Elementary School,97.60%
5,Annie Keller Elementary Gifted Magnet School,97.50%
6,Andrew Jackson Elementary Language Academy,97.40%
7,Lenart Elementary Regional Gifted Center,97.40%
8,Disney II Magnet School,97.30%
9,John H Vanderpoel Elementary Magnet School,97.20%


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from CHICAGO_PUBLIC_SCHOOLS_DATA \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [13]:
# %%sql
# select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE
# from CHICAGO_PUBLIC_SCHOOLS_DATA
# order by AVERAGE_STUDENT_ATTENDANCE asc nulls last
# limit 5;

query = """
select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE
from CHICAGO_PUBLIC_SCHOOLS_DATA
order by AVERAGE_STUDENT_ATTENDANCE asc nulls last
limit 5;
"""

cur.execute(query)

rows = cur.fetchall()

column_names = [desc[0] for desc in cur.description]

lowest_attendance_df = pd.DataFrame(rows, columns=column_names)

display(lowest_attendance_df)

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
0,Richard T Crane Technical Preparatory High School,57.90%
1,Barbara Vick Early Childhood & Family Center,60.90%
2,Dyett High School,62.50%
3,Wendell Phillips Academy High School,63.00%
4,Orr Academy High School,66.30%


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from CHICAGO_PUBLIC_SCHOOLS_DATA \
     order by Average_Student_Attendance \
     LIMIT 5

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [14]:
# %%sql
# select NAME_OF_SCHOOL as name_of_school, replace(AVERAGE_STUDENT_ATTENDANCE, '%', '') as avg_student_attendance
# from CHICAGO_PUBLIC_SCHOOLS_DATA
# order by AVERAGE_STUDENT_ATTENDANCE asc nulls last
# limit 5;

query = """
select NAME_OF_SCHOOL as name_of_school, replace(AVERAGE_STUDENT_ATTENDANCE, '%', '') as avg_student_attendance
from CHICAGO_PUBLIC_SCHOOLS_DATA
order by AVERAGE_STUDENT_ATTENDANCE asc nulls last
limit 5;
"""

cur.execute(query)

rows = cur.fetchall()

column_names = [desc[0] for desc in cur.description]

lowest_attendance_df = pd.DataFrame(rows, columns=column_names)

display(lowest_attendance_df)

,name_of_school,avg_student_attendance
0,Richard T Crane Technical Preparatory High School,57.90
1,Barbara Vick Early Childhood & Family Center,60.90
2,Dyett High School,62.50
3,Wendell Phillips Academy High School,63.00
4,Orr Academy High School,66.30


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''

-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from CHICAGO_PUBLIC_SCHOOLS_DATA \
     order by Average_Student_Attendance \
     LIMIT 5

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [15]:
# %%sql
# select NAME_OF_SCHOOL as name_of_school, cast(AVERAGE_STUDENT_ATTENDANCE as decimal(3, 2)) as avg_student_attendance
# from CHICAGO_PUBLIC_SCHOOLS_DATA
# where AVERAGE_STUDENT_ATTENDANCE < 70
# order by AVERAGE_STUDENT_ATTENDANCE asc;

query = """
select 
    NAME_OF_SCHOOL as name_of_school,
    AVERAGE_STUDENT_ATTENDANCE as avg_student_attendance
from CHICAGO_PUBLIC_SCHOOLS_DATA
where cast(replace(AVERAGE_STUDENT_ATTENDANCE, '%', '') as decimal(3, 2)) < 70
order by AVERAGE_STUDENT_ATTENDANCE asc;
"""

cur.execute(query)
rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=column_names)
display(df)

,name_of_school,avg_student_attendance
0,Richard T Crane Technical Preparatory High School,57.90%
1,Barbara Vick Early Childhood & Family Center,60.90%
2,Dyett High School,62.50%
3,Wendell Phillips Academy High School,63.00%
4,Orr Academy High School,66.30%
5,Manley Career Academy High School,66.80%
6,Chicago Vocational Career Academy High School,68.80%
7,Roberto Clemente Community Academy High School,69.60%


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)

-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from CHICAGO_PUBLIC_SCHOOLS_DATA \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [16]:
# %%sql
# select
#   COMMUNITY_AREA_NAME,
#   sum(COLLEGE_ENROLLMENT) as total_college_enrollment
# from
#   CHICAGO_PUBLIC_SCHOOLS_DATA
# group by
#   COMMUNITY_AREA_NAME
# order by total_college_enrollment desc;

query = """
SELECT
    COMMUNITY_AREA_NAME,
    SUM(COLLEGE_ENROLLMENT) AS total_college_enrollment
FROM 
    CHICAGO_PUBLIC_SCHOOLS_DATA
GROUP BY 
    COMMUNITY_AREA_NAME
ORDER BY 
    total_college_enrollment DESC;
"""

cur.execute(query)
rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
enrollment_df = pd.DataFrame(rows, columns=column_names)
display(enrollment_df)

,COMMUNITY_AREA_NAME,total_college_enrollment
0,SOUTH LAWNDALE,14793
1,BELMONT CRAGIN,14386
2,AUSTIN,10933
3,GAGE PARK,9915
4,BRIGHTON PARK,9647
...,...,...
72,LOOP,871
73,OHARE,786
74,BURNSIDE,549
75,FULLER PARK,531


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from CHICAGO_PUBLIC_SCHOOLS_DATA \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [17]:
# %%sql
# select
#   COMMUNITY_AREA_NAME,
#   sum(COLLEGE_ENROLLMENT) as total_college_enrollment
# from
#   CHICAGO_PUBLIC_SCHOOLS_DATA
# group by
#   COMMUNITY_AREA_NAME
# order by total_college_enrollment asc
# limit 5;

query = """
SELECT
    COMMUNITY_AREA_NAME,
    SUM(COLLEGE_ENROLLMENT) AS total_college_enrollment
FROM 
    CHICAGO_PUBLIC_SCHOOLS_DATA
GROUP BY 
    COMMUNITY_AREA_NAME
ORDER BY 
    total_college_enrollment asc
limit 5;
"""

cur.execute(query)
rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
enrollment_df = pd.DataFrame(rows, columns=column_names)
display(enrollment_df)

,COMMUNITY_AREA_NAME,total_college_enrollment
0,OAKLAND,140
1,FULLER PARK,531
2,BURNSIDE,549
3,OHARE,786
4,LOOP,871


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from CHICAGO_PUBLIC_SCHOOLS_DATA \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   LIMIT 5 

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [18]:
# %%sql
# select 
#     NAME_OF_SCHOOL, SAFETY_SCORE
# from
#     CHICAGO_PUBLIC_SCHOOLS_DATA
# order by
#     SAFETY_SCORE asc nulls last
# limit 5;

query = """
select
    NAME_OF_SCHOOL, SAFETY_SCORE
from
    CHICAGO_PUBLIC_SCHOOLS_DATA
order by
    SAFETY_SCORE asc nulls last
limit 5;
"""

cur.execute(query)
rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
lowest_safety_score = pd.DataFrame(rows, columns=column_names)
display(lowest_safety_score)

,NAME_OF_SCHOOL,SAFETY_SCORE
0,Edmond Burke Elementary School,1.0
1,Luke O'Toole Elementary School,5.0
2,George W Tilton Elementary School,6.0
3,Foster Park Elementary School,11.0
4,Emil G Hirsch Metropolitan High School,13.0


Double-click **here** for the solution.

<!-- Solution SQLite:

%sql SELECT name_of_school, safety_score \
FROM CHICAGO_PUBLIC_SCHOOLS_DATA  where safety_score !='None' \
ORDER BY safety_score \
LIMIT 5

-->




### Problem 11

##### Get the hardship index for the community area of the school which has College Enrollment of 4368


In [19]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")

df.to_sql("CENSUS_DATA", con=con, if_exists="replace", index=False, method="multi")

cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cur.fetchall()
print("Tables in DB:")
for i, table in enumerate(tables, 1):
    print(f"{i}. {table[0]}")

Tables in DB:
1. CHICAGO_PUBLIC_SCHOOLS_DATA
2. CENSUS_DATA


In [20]:
# %%sql
# select 
#     cd.HARDSHIP_INDEX
# from 
#     CENSUS_DATA cd
# join 
#     CHICAGO_PUBLIC_SCHOOLS_DATA cps
#     on cd.COMMUNITY_AREA_NUMBER = cps.COMMUNITY_AREA_NUMBER
# where 
#     cps.COLLEGE_ENROLLMENT = 4368;

query = """
select 
    cd.HARDSHIP_INDEX
from 
    CENSUS_DATA cd
join 
    CHICAGO_PUBLIC_SCHOOLS_DATA cps
    on cd.COMMUNITY_AREA_NUMBER = cps.COMMUNITY_AREA_NUMBER
where 
    cps.COLLEGE_ENROLLMENT = 4368;
"""

cur.execute(query)
rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
hardship_df = pd.DataFrame(rows, columns=column_names)
display(hardship_df)

,HARDSHIP_INDEX
0,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist. If the CHICAGO_SOCIOECONOMIC_DATA table does not exists, please execute the following code beforehand.

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")


%%sql 
select hardship_index from CENSUS_DATA CD, CHICAGO_PUBLIC_SCHOOLS_DATA CPS 
where CD.community_area_number = CPS.community_area_number 
and college_enrollment = 4368


-->


### Problem 12

##### Get the hardship index for the community area which has the highest value for College Enrollment


In [21]:
# %%sql
# select
#     cps.COMMUNITY_AREA_NUMBER, cps.COMMUNITY_AREA_NAME, cd.HARDSHIP_INDEX
# from
#     CHICAGO_PUBLIC_SCHOOLS_DATA cps
# join
#     CENSUS_DATA cd
#     on cd.COMMUNITY_AREA_NUMBER = cps.COMMUNITY_AREA_NUMBER
# where
#     cps.COLLEGE_ENROLLMENT = (
#         select max(COLLEGE_ENROLLMENT)
#         from CHICAGO_PUBLIC_SCHOOLS_DATA
#     );

query = """
select
    cps.COMMUNITY_AREA_NUMBER, cps.COMMUNITY_AREA_NAME, cd.HARDSHIP_INDEX
from
    CHICAGO_PUBLIC_SCHOOLS_DATA cps
join
    CENSUS_DATA cd
    on cd.COMMUNITY_AREA_NUMBER = cps.COMMUNITY_AREA_NUMBER
where
    cps.COLLEGE_ENROLLMENT = (
        select max(COLLEGE_ENROLLMENT)
        from CHICAGO_PUBLIC_SCHOOLS_DATA
    );
"""

cur.execute(query)
rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
hardship_df = pd.DataFrame(rows, columns=column_names)
display(hardship_df)

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
0,5,NORTH CENTER,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist. If the CHICAGO_SOCIOECONOMIC_DATA table does not exists, please execute the following code beforehand.

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

%sql select community_area_number, community_area_name, hardship_index from CENSUS_DATA \
   where community_area_number in \
   ( select community_area_number from CHICAGO_PUBLIC_SCHOOLS_DATA order by college_enrollment desc limit 1 )
-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01" target="_blank">Rav Ahuja</a>
<!--
## Change Log

| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                        |
| ----------------- | ------- | ----------------- | ----------------------------------------- |
| 2022-03-04        | 2.2     | Lakshmi Holla     | Made changes in markdown cells |
| 2020-11-27        | 2.1     | Sannareddy Ramesh | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya           | Moved lab to course repo in GitLab        |
-->
<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
